In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Time-series forecasting with Deep Learning
Modified script based on [How to use Keras TimeseriesGenerator for time series data](https://www.dlology.com/blog/how-to-use-keras-timeseriesgenerator-for-time-series-data/)

#### Instalation
`mpld3` enables zooming of the matplotlib and interactive data visualizations on Jupyter notebook.

Install it with `pip install mpld3`

Also `pip install pydot`

In [ ]:
import mpld3
mpld3.enable_notebook()

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from sklearn.preprocessing import MinMaxScaler, PowerTransformer, QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

import math
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
plt.rcParams["figure.figsize"] = (12,8)

import seaborn as sns
sns.set(context='paper', style='whitegrid', color_codes=True)   
sns.set_palette(sns.color_palette(["#017b92", "#f97306", "#ff0000"]))  # ["green", "orange", "red"] 

#### Functions definitions

In [ ]:
# get all targets y from a TimeseriesGenerator instance.
def get_y_from_generator(gen):
    y = None
    for i in range(len(gen)):
        batch_y = gen[i][1]
        if y is None:
            y = batch_y
        else:
            y = np.append(y, batch_y)
    y = y.reshape((-1,1))
    print(y.shape)
    return y

# compute the match score of two binary numpy arrays.
def binary_accuracy(a, b):
    assert len(a) == len(b)
    return (a == b).sum() / len(a)    

#### Load data
DJI, AAPL, AMAZN close index/price during 2015/09/19-2018/09/18
- URL [YAHOO finance](https://finance.yahoo.com/quote/AMZN/history?period1=1442592000&period2=1537286400&interval=1d&filter=history&frequency=1d).

In [ ]:
dataset_DJI = pd.read_csv('./data/DJI_20150919-20180918.csv', usecols=[4]).values.astype('float32')
dataset = dataset_DJI
print(dataset.shape)
plt.plot(dataset_DJI)


## Single time series as input
#### Data normalization

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

#### Data splitting and transformation

In [ ]:
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

look_back = 10
train_data_gen = TimeseriesGenerator(train, 
                                     train,
                                     length=look_back, 
                                     sampling_rate=1,
                                     stride=1,
                                     batch_size=1
                                    )
test_data_gen = TimeseriesGenerator(test, 
                                    test,
                                    length=look_back, 
                                    sampling_rate=1,
                                    stride=1,
                                    batch_size=1
                                   )

#### Training

In [ ]:
x = Input(shape=(look_back, 1))
h = LSTM(units=200)(x)
y = Dense(units=1, activation='sigmoid')(h)
model = Model(inputs=x, outputs=y)
print(model.summary())

# compile model
opt = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse', 'mae'])

In [ ]:
model.fit(train_data_gen, epochs=100)

In [ ]:
model.evaluate(test_data_gen)

In [ ]:
trainPredict = model.predict(train_data_gen)
trainPredict.shape

In [ ]:
testPredict = model.predict(test_data_gen)
testPredict.shape

#### Invert transformation

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)

In [ ]:
trainY = get_y_from_generator(train_data_gen)
testY = get_y_from_generator(test_data_gen)

In [ ]:
trainY = scaler.inverse_transform(trainY)
testY = scaler.inverse_transform(testY)

#### Model evaluation

In [ ]:
# RMSE
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:, 0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#### Plotting

In [ ]:
begin = train_size + look_back
end = begin + len(testPredict)

testYPlot = np.empty_like(dataset_DJI)
testYPlot[:, :] = np.nan
testYPlot[begin:end, :] = testY

testPredictPlot = np.empty_like(dataset_DJI)
testPredictPlot[:, :] = np.nan
testPredictPlot[begin:end, :] = testPredict

In [ ]:
# plot baseline and predictions
plt.plot(dataset_DJI)
plt.plot(testYPlot)
plt.plot(testPredictPlot)
plt.show()